In [1]:
# Import Packages

from os.path import exists
import pickle
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

time_start = pd.Timestamp.now()

## Obtain Election Data

In [6]:
# check to see if the data has been captured
# if True then move on, if False then grab data and capture information

if exists('./data/dataPickle'):
    print('Data exists in the dataPickle file')
else:
    # Setup pickle file to store scraped tables

    # create and open pickle file 
    election_scraped_file = open('./data/dataPickle', 'ab')

    # extract datat from source
    election_scrape = pd.read_html('https://alex.github.io/nyt-2020-election-scraper/all-state-changes.html')

    # add content to pickle file
    pickle.dump(election_scrape, election_scraped_file)

    # closse pickle file
    election_scraped_file.close()
    
    print('Data file not found.  Data is now stored in the dataPickle file')


Data exists in the dataPickle file


In [4]:
stored_pickle_file = open('./data/dataPickle', 'rb')
election_results = pickle.load(stored_pickle_file)


## Create summary table

In [5]:
# extract out header information that contains total votes, electoral votes, percentages, and winner
summary_table = []

for state in election_results:
    stringState = state.columns.get_level_values(0)[0]
    d = stringState.split("  ",2)
    state_name = d[0].split("(")[0].strip()
    electoral_votes = d[0].split("(EV: ")[1].replace(")","").strip()
    s = d[1].replace(",","")
    lead_votes, lead_percent, trail_votes, trail_percent = [float(i) for i in re.findall(r'([0-9]+\.?\d)', s)]
    leader, trailer = re.findall(r'([A-Z][a-z]+)',s)
    dict = {
        'State': state_name, 
        'Electoral Votes': int(electoral_votes), 
        'Leader': leader, 
        'Leader Total Votes': lead_votes, 
        'Leader Vote Percent': lead_percent, 
        'Trailer': trailer, 
        'Trailer Total Votes': trail_votes, 
        'Trailer Vote Percent': trail_percent
    }
    summary_table.append(dict)
summary_df = pd.DataFrame(summary_table)
summary_df.head()

,State,Electoral Votes,Leader,Leader Total Votes,Leader Vote Percent,Trailer,Trailer Total Votes,Trailer Vote Percent
0,Alabama,9,Trump,1441170.0,62.0,Biden,849624.0,36.6
1,Alaska,3,Trump,189951.0,52.8,Biden,153778.0,42.8
2,Arizona,11,Biden,1672143.0,49.4,Trump,1661686.0,49.1
3,Arkansas,6,Trump,760647.0,62.4,Biden,423932.0,34.8
4,California,55,Biden,11109764.0,63.5,Trump,6005961.0,34.3


In [7]:
# Check data types for accuracy
summary_df.dtypes

State                    object
Electoral Votes           int64
Leader                   object
Leader Total Votes      float64
Leader Vote Percent     float64
Trailer                  object
Trailer Total Votes     float64
Trailer Vote Percent    float64
dtype: object

In [8]:
# Store summary table as csv
summary_df.to_csv('./data/summary.csv', index=False)

## Make election results table

In [9]:
def extract_batch(s, index):
    string = re.findall(r'([0-9]+\.?\d)', s)
    try:
        strval = string[index]
    except:
        strval = None
        
    return strval

In [10]:
# election_records = pd.read_html('https://alex.github.io/nyt-2020-election-scraper/all-state-changes.html')
frames = []

for state in election_results:
    stringState = state.columns.get_level_values(0)[0]
    d = stringState.split("  ",2)
    state_name = d[0].split("(")[0].strip()
    df = state
    df.columns = df.columns.droplevel(0)
    df['State'] = state_name
    df['Leader Batch Breakdown'] = df['Batch Breakdown'].apply(lambda x: extract_batch(str(x), 0) )
    df['Trailer Batch Breakdown'] = df['Batch Breakdown'].apply(lambda x: extract_batch(str(x), 1) )
    df['Trailer Batch Trend'] = df['Batch Trend'].apply(lambda x: extract_batch(str(x), 0) )
    df['Trailer Hurdle'] = df['Hurdle'].apply(lambda x: extract_batch(str(x), 0) )
    df.drop('Unnamed: 8_level_1', axis='columns', inplace=True)
    df.drop('Batch Breakdown', axis='columns', inplace=True)
    df.drop(['Batch Trend', 'Hurdle'], axis='columns', inplace=True)
    frames.append(df)

result_df = pd.concat(frames)

In [12]:
# instead of using .dtypes, use .info() to see aditional information
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3406 entries, 0 to 4
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Timestamp                3406 non-null   object
 1   In The Lead              3406 non-null   object
 2   Vote Margin              3406 non-null   int64 
 3   Votes Remaining (est.)   3406 non-null   object
 4   Change                   3406 non-null   object
 5   State                    3406 non-null   object
 6   Leader Batch Breakdown   2886 non-null   object
 7   Trailer Batch Breakdown  2886 non-null   object
 8   Trailer Batch Trend      3298 non-null   object
 9   Trailer Hurdle           3166 non-null   object
dtypes: int64(1), object(9)
memory usage: 292.7+ KB


In [ ]:
# Store batch vote records for each state
# don't include the index column
result_df.to_csv('./data/raw_records.csv', index=False)

## Dataset Review

The data in the columns is not very helpful in its current form.  I need the geospacial-temporal data to have a highly versatile dataset.
Since this data is mostly about batches of votes but has some faulty information, I need to calculate the vote counting trends based on the final vote tally and this batch data.  Verifying the validity of the data is necessary.

The columns that I need are time, candidate, batch votes, state, district, maybe a note if any of the records were estimated or not.

Here are somet things I think that should be checked:
* Do the zero values in the 'Change' column need to be removed?  Is there anything else wrong with this data?
* Do the 'Unknown' values in the 'Change' column need to be removed?  
* Can I estimate the 'Change' column zero and 'Unknown' values from the change in the margin votes and the percent of the leader batch votes?
* Is the change in margin between batches smaller than the batch size?  This must be true and values that violate this are likely faulty.

In the end, the batch votes need to be filtered for faulty data and the individual batch candidate votes should be calculated from the existing data.

In [17]:
# Create two new columns, convert 3 columns to numeric type
result_df['Margin_change_estimate']=""
result_df['Type']=""
result_df[['Margin_change_estimate','Leader Batch Breakdown', 'Trailer Batch Breakdown']]=result_df[['Margin_change_estimate', 'Leader Batch Breakdown', 'Trailer Batch Breakdown']].apply(pd.to_numeric)

# renumber index and discard current index
result_df.reset_index(drop=True, inplace=True)

# view updated column info
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3406 entries, 0 to 3405
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Timestamp                3406 non-null   object 
 1   In The Lead              3406 non-null   object 
 2   Vote Margin              3406 non-null   int64  
 3   Votes Remaining (est.)   3406 non-null   object 
 4   Change                   3406 non-null   object 
 5   State                    3406 non-null   object 
 6   Leader Batch Breakdown   2886 non-null   float64
 7   Trailer Batch Breakdown  2886 non-null   float64
 8   Trailer Batch Trend      3298 non-null   object 
 9   Trailer Hurdle           3166 non-null   object 
 10  Margin_change_estimate   0 non-null      float64
 11  Type                     3406 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 319.4+ KB


## Zero Data Review

In [38]:
# Check Zero Data - is it real or is it flawed
# Note - searching for a string bc I can not convert to numeric directly until "Unknowns" are removed
result_df[result_df["Change"] == "0"]
# shows what looks like data without any percent breakdowns of the batch

,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type
5,2020-11-11 03:23:42 UTC,Trump,590686,13386,0,Alabama,NaN,NaN,44.9,2287.6,NaN,
13,2020-11-04 13:28:22 UTC,Trump,618344,93031,0,Alabama,NaN,NaN,None,387.0,NaN,
14,2020-12-02 19:04:28 UTC,Trump,36173,31816,0,Alaska,NaN,NaN,49.4,109.5,NaN,
15,2020-12-02 17:18:21 UTC,Trump,36173,31816,0,Alaska,NaN,NaN,49.4,109.5,NaN,
16,2020-12-02 16:33:22 UTC,Trump,36173,31816,0,Alaska,NaN,NaN,49.4,109.5,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...
3358,2020-11-10 13:17:25 UTC,Trump,307807,6992,0,West Virginia,NaN,NaN,34.6,2288.6,NaN,
3359,2020-11-09 23:16:41 UTC,Trump,307807,6992,0,West Virginia,NaN,NaN,34.6,2288.6,NaN,
3372,2020-11-04 13:28:22 UTC,Trump,307126,8526,0,West Virginia,NaN,NaN,None,1881.7,NaN,
3399,2020-11-04 17:33:32 UTC,Biden,20697,592,0,Wisconsin,NaN,NaN,None,1828.6,NaN,


In [39]:
# check to see if any of the percent columns have bad data (values greater than 100%)
result_df[(result_df["Change"] == "0") & ( (result_df["Leader Batch Breakdown"] >100) | (result_df["Trailer Batch Breakdown"] >100) )]
# Note: there are no records of Change data outside noral percentages

,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type


In [40]:
# check to see if the vote percents add up to 100%
result_df[(result_df["Change"] == "0") & (result_df["Leader Batch Breakdown"] + result_df["Trailer Batch Breakdown"] == 100) ]

# Note:  none of the values add up to 100 so all values can be thrown out as bad data since individual percents are not greater than 100 and they don't add up to 100

,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type


## Unknown Data Review

In [41]:
# Check if Unknowns should be thrown out
unknown_df = result_df[result_df['Change']=="Unknown"]
print(f"Number of Unknowns: {len(unknown_df)}")
unknown_df.head()

Number of Unknowns: 113


,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type
0,2021-01-05 15:21:52 UTC,Trump,591546,4,Unknown,Alabama,9.1,109.1,58.5,7499241.2,NaN,
7,2020-11-06 18:40:11 UTC,Trump,596046,26070,Unknown,Alabama,5.9,105.9,49.1,1209.4,NaN,
22,2020-11-25 01:59:12 UTC,Trump,36355,34858,Unknown,Alaska,6.8,106.8,48.2,104.5,NaN,
157,2020-11-16 18:16:52 UTC,Trump,338005,7175,Unknown,Arkansas,74.0,26.0,36.0,2474.0,NaN,
192,2020-11-09 17:17:21 UTC,Trump,338978,9790,Unknown,Arkansas,48.6,51.4,29.2,1831.7,NaN,


In [42]:
# I can use the percents and change in vote margin to calculate the votes for each candidate.  This is not an ideal way since 
# percents are rounded and induce error.

# check if percents add to one hundred percent
result_df[ (result_df['Change']=="Unknown") & (result_df['Leader Batch Breakdown'] + result_df['Trailer Batch Breakdown'] == 100) ]
# These 97 rows can be used to calculate the Unknown value

,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type
157,2020-11-16 18:16:52 UTC,Trump,338005,7175,Unknown,Arkansas,74.0,26.0,36.0,2474.0,NaN,
192,2020-11-09 17:17:21 UTC,Trump,338978,9790,Unknown,Arkansas,48.6,51.4,29.2,1831.7,NaN,
193,2020-11-09 16:41:11 UTC,Trump,338874,5916,Unknown,Arkansas,75.7,24.3,33.3,2997.4,NaN,
197,2020-11-06 19:30:23 UTC,Trump,337918,11131,Unknown,Arkansas,75.7,24.3,33.7,1612.1,NaN,
198,2020-11-06 19:13:48 UTC,Trump,340363,6134,Unknown,Arkansas,48.7,51.3,27.3,2905.2,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...
3369,2020-11-08 16:45:34 UTC,Trump,307108,8464,Unknown,West Virginia,65.3,34.7,56.5,1895.0,NaN,
3373,2020-12-08 01:37:07 UTC,Biden,20608,76,Unknown,Wisconsin,61.9,38.1,64.4,13844.7,NaN,
3381,2020-11-16 19:31:33 UTC,Biden,20546,202,Unknown,Wisconsin,0.0,100.0,67.6,5224.5,NaN,
3389,2020-11-12 01:26:41 UTC,Biden,20798,321,Unknown,Wisconsin,0.7,99.3,35.8,3346.2,NaN,


In [43]:
result_df[ (result_df['Change']=="Unknown") & (result_df['Leader Batch Breakdown'] + result_df['Trailer Batch Breakdown'] != 100) ]
# These 16 rows have faulty change and percent breakdowns so these batch records should be thrown out.

,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type
0,2021-01-05 15:21:52 UTC,Trump,591546,4,Unknown,Alabama,9.1,109.1,58.5,7499241.2,NaN,
7,2020-11-06 18:40:11 UTC,Trump,596046,26070,Unknown,Alabama,5.9,105.9,49.1,1209.4,NaN,
22,2020-11-25 01:59:12 UTC,Trump,36355,34858,Unknown,Alaska,6.8,106.8,48.2,104.5,NaN,
899,2020-11-09 21:52:34 UTC,Biden,711793,535553,Unknown,Illinois,38.3,138.3,21.6,117.7,NaN,
993,2020-11-11 14:59:03 UTC,Trump,138793,3227,Unknown,Iowa,114.3,14.3,39.1,2244.8,NaN,
1629,2020-11-25 14:47:14 UTC,Biden,233039,90020,Unknown,Minnesota,37.0,137.0,52.7,182.5,NaN,
1632,2020-11-25 14:10:14 UTC,Biden,232802,94785,Unknown,Minnesota,300.0,400.0,52.7,175.7,NaN,
1813,2020-12-06 14:16:59 UTC,Trump,217366,12735,Unknown,Mississippi,52.4,152.4,69.3,915.9,NaN,
1823,2020-11-17 02:21:52 UTC,Trump,236423,67085,Unknown,Mississippi,736.4,836.4,54.3,228.8,NaN,
1878,2020-12-09 14:21:38 UTC,Trump,465722,1422,Unknown,Missouri,13.3,113.3,59.6,16724.3,NaN,


## Change in Margin Data Review

In [44]:
# For each batch added the margin of the votes can not increase/decrease more than the Change columns
# key for this is to enure we are only looking at data for each state because if the last row has an unknown then this calculation will not work

# create a copy of the data so I can change the index a bit
# I added the 'Margin_change_estimate' column in the original dataframe so the df are identical
df = result_df.copy()

# preserve the original index in the groupby (next step)
df.reset_index(inplace=True)

# first find the last rows of each state
last_row_list = list(df.groupby('State').last()['index'])
print(last_row_list)

[13, 39, 125, 211, 355, 428, 559, 565, 575, 624, 850, 854, 863, 933, 984, 1060, 1153, 1210, 1225, 1321, 1419, 1545, 1622, 1810, 1873, 1890, 1910, 1935, 1981, 2046, 2191, 2216, 2298, 2364, 2401, 2443, 2448, 2484, 2820, 2832, 2896, 2938, 2945, 3001, 3055, 3085, 3191, 3341, 3372, 3400, 3405]


In [45]:
# go through original df and create the change in margin calculation except on last rows of each state
for i in range(len(result_df)-1):
    if i not in last_row_list:
        vote_margin_now = result_df.iloc[i,2]
        vote_margin_prev = result_df.iloc[i+1, 2]
        result_df.iloc[i, 10] = abs(vote_margin_now - vote_margin_prev)

In [47]:
# how many NaN are in the Margin_change_estimate column - these are last row values that lack data to calculate a change value
# remove all of these values from the df
result_df[ result_df['Margin_change_estimate'].isnull() ]
# Actually, lets leave them in the df since these are the last row values

,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type
13,2020-11-04 13:28:22 UTC,Trump,618344,93031,0,Alabama,NaN,NaN,None,387.0,NaN,
39,2020-11-04 13:28:22 UTC,Trump,35241,258871,0,Alaska,NaN,NaN,None,57.1,NaN,
125,2020-11-04 13:28:22 UTC,Biden,130665,746304,0,Arizona,NaN,NaN,None,58.9,NaN,
211,2020-11-04 13:28:22 UTC,Trump,335478,21014,0,Arkansas,NaN,NaN,None,871.5,NaN,
355,2020-11-04 13:28:22 UTC,Biden,3829646,5647390,0,California,NaN,NaN,None,84.7,NaN,
428,2020-11-04 13:28:22 UTC,Biden,410357,333648,0,Colorado,NaN,NaN,None,113.2,NaN,
559,2020-11-04 13:28:22 UTC,Biden,294420,237831,0,Connecticut,NaN,NaN,None,112.9,NaN,
565,2020-11-04 13:28:22 UTC,Biden,95573,1667,0,Delaware,NaN,NaN,None,2957.8,NaN,
575,2020-11-04 13:28:22 UTC,Biden,199740,113818,0,District of Columbia,NaN,NaN,None,140.0,NaN,
624,2020-11-04 13:28:22 UTC,Trump,377023,36016,0,Florida,NaN,NaN,None,578.3,NaN,


In [48]:
# Check data types before continuing
result_df.dtypes

Timestamp                   object
In The Lead                 object
Vote Margin                  int64
Votes Remaining (est.)      object
Change                      object
State                       object
Leader Batch Breakdown     float64
Trailer Batch Breakdown    float64
Trailer Batch Trend         object
Trailer Hurdle              object
Margin_change_estimate     float64
Type                        object
dtype: object

In [49]:
## Need to convert the timestamp to a date and convert the Change column to integer - but I think this needs done after the unknowns are removed.
result_df[['Timestamp']]=result_df[['Timestamp']].apply(pd.to_datetime)  ## could pass in arguments to apply like:  .apply(pd.to_datetime, args=(format = ) )
result_df.dtypes


Timestamp                  datetime64[ns, UTC]
In The Lead                             object
Vote Margin                              int64
Votes Remaining (est.)                  object
Change                                  object
State                                   object
Leader Batch Breakdown                 float64
Trailer Batch Breakdown                float64
Trailer Batch Trend                     object
Trailer Hurdle                          object
Margin_change_estimate                 float64
Type                                    object
dtype: object

In [50]:
# Throw out Unknowns where the percents do not add to 100%
bad_unknowns = list(result_df[ (result_df['Change']=="Unknown") & (result_df['Leader Batch Breakdown'] + result_df['Trailer Batch Breakdown'] != 100) ].index)
print(bad_unknowns)

[0, 7, 22, 899, 993, 1629, 1632, 1813, 1823, 1878, 2441, 2974, 3096, 3097, 3098, 3383]


In [51]:
# drop bad Unknown values that can not be replaced with the help of percent values
result_df.drop(bad_unknowns, inplace=True)

## Remove Zero Values (not removing last row zeros)

In [ ]:
# remove zero values but modify below so only non-last values are removed.
# zero_check = result_df[(result_df['Change'] == 'Unknown') | (result_df['Change'] == "0")]
results_excludes_last_row = result_df[~result_df.index.isin(last_row_list)]
zero_check = results_excludes_last_row[(results_excludes_last_row['Change'] == "0")]
zero_values = list(zero_check.index)
result_df.drop(zero_values, inplace=True)

## Calculate Significant Columns
* Time Stamp
* Leader
* Vote Margin
* Votes Remaining
* Batch Total
* Batch Votes Winner
* Batch Votes Loser
* Accum. Votes Winner
* Accum. Votes Loser
* Trailer Batch Trend
* Hurdle Percent Needed

In [19]:
# checked multiple rows to see if 'Change' column  is useable
result_df.head()

,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type
0,2021-01-05 15:21:52 UTC,Trump,591546,4,Unknown,Alabama,9.1,109.1,58.5,7499241.2,NaN,
1,2020-12-04 21:12:00 UTC,Trump,591520,Unknown,22,Alabama,9.1,109.1,58.5,None,NaN,
2,2020-11-23 20:33:25 UTC,Trump,591546,4,2476,Alabama,44.2,55.8,58.5,7499241.2,NaN,
3,2020-11-23 20:11:23 UTC,Trump,591832,2521,5894,Alabama,50.6,49.4,55.0,11954.5,NaN,
4,2020-11-23 19:48:16 UTC,Trump,591760,8534,4792,Alabama,61.2,38.8,47.8,3566.2,NaN,


In [59]:
# replace some of the unknown values
batch_leader_votes = result_df['Leader Batch Breakdown']
unknown_check = result_df[(result_df['Change'] == 'Unknown') & ( (batch_leader_votes >= 0) | (batch_leader_votes <= 100) ) ].copy()
for i in range(len(unknown_check)):
    
    raw_value_percent = unknown_check.iloc[i,6]
    winner_batch_percent = float(raw_value_percent)/100
    margin_change_estimate = unknown_check.iloc[i, 10]

#     new_change = round((vote_margin_now - vote_margin_prev)/(winner_batch_percent),0)
    if (raw_value_percent != 0.0):
        new_change = round(margin_change_estimate/winner_batch_percent,0)
    else:
        new_change = round(margin_change_estimate,0)
        print(f"Can not calculate vote margin for row {i}")
        
    unknown_check.iloc[i, 4] = new_change
    unknown_check.iloc[i, 11] = "updated"



# I should store these in a dataframe for future analysis


Can not calculate vote margin for row 19
Can not calculate vote margin for row 26
Can not calculate vote margin for row 50
Can not calculate vote margin for row 51
Can not calculate vote margin for row 88
Can not calculate vote margin for row 89
Can not calculate vote margin for row 94


In [60]:
unknown_check.head(30)

,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type
157,2020-11-16 18:16:52+00:00,Trump,338005,7175,753.0,Arkansas,74.0,26.0,36.0,2474.0,557.0,updated
192,2020-11-09 17:17:21+00:00,Trump,338978,9790,214.0,Arkansas,48.6,51.4,29.2,1831.7,104.0,updated
193,2020-11-09 16:41:11+00:00,Trump,338874,5916,3230.0,Arkansas,75.7,24.3,33.3,2997.4,2445.0,updated
197,2020-11-06 19:30:23+00:00,Trump,337918,11131,3230.0,Arkansas,75.7,24.3,33.7,1612.1,2445.0,updated
198,2020-11-06 19:13:48+00:00,Trump,340363,6134,207.0,Arkansas,48.7,51.3,27.3,2905.2,101.0,updated
199,2020-11-06 18:51:26+00:00,Trump,340262,2257,3000.0,Arkansas,100.0,0.0,36.8,7807.4,3000.0,updated
316,2020-11-09 17:17:21+00:00,Biden,4496538,3092599,7694.0,California,77.8,22.2,35.9,124.3,5986.0,updated
359,2020-11-25 02:25:12+00:00,Biden,439717,330,49.0,Colorado,66.7,33.3,42.4,68523.8,33.0,updated
372,2020-11-16 03:32:50+00:00,Biden,439868,2119,363.0,Colorado,42.7,57.3,42.4,10717.3,155.0,updated
435,2020-11-25 02:48:20+00:00,Biden,364750,1570,3287.0,Connecticut,12.9,87.1,37.8,11849.3,424.0,updated


In [61]:
restored = pd.concat([result_df, unknown_check])

In [62]:
filtered_df = restored[~restored.index.duplicated(keep='last')]
filtered_df.head()

,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type
1,2020-12-04 21:12:00+00:00,Trump,591520,Unknown,22,Alabama,9.1,109.1,58.5,None,26.0,
2,2020-11-23 20:33:25+00:00,Trump,591546,4,2476,Alabama,44.2,55.8,58.5,7499241.2,286.0,
3,2020-11-23 20:11:23+00:00,Trump,591832,2521,5894,Alabama,50.6,49.4,55.0,11954.5,72.0,
4,2020-11-23 19:48:16+00:00,Trump,591760,8534,4792,Alabama,61.2,38.8,47.8,3566.2,1074.0,
6,2020-11-06 22:26:37+00:00,Trump,590686,13386,12548,Alabama,28.6,71.4,44.9,2287.6,5360.0,


In [63]:
filtered_df[ filtered_df['State'] == 'Alabama']

,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type
1,2020-12-04 21:12:00+00:00,Trump,591520,Unknown,22,Alabama,9.1,109.1,58.5,None,26.0,
2,2020-11-23 20:33:25+00:00,Trump,591546,4,2476,Alabama,44.2,55.8,58.5,7499241.2,286.0,
3,2020-11-23 20:11:23+00:00,Trump,591832,2521,5894,Alabama,50.6,49.4,55.0,11954.5,72.0,
4,2020-11-23 19:48:16+00:00,Trump,591760,8534,4792,Alabama,61.2,38.8,47.8,3566.2,1074.0,
6,2020-11-06 22:26:37+00:00,Trump,590686,13386,12548,Alabama,28.6,71.4,44.9,2287.6,5360.0,
8,2020-11-04 21:12:19+00:00,Trump,595572,25662,10,Alabama,30.0,70.0,49.4,1226.9,4.0,
9,2020-11-04 21:00:38+00:00,Trump,595576,25815,2938,Alabama,59.2,40.8,49.5,1219.9,542.0,
10,2020-11-04 20:28:27+00:00,Trump,595034,28763,54,Alabama,1870.4,1970.4,53.5,1099.0,2074.0,
11,2020-11-04 20:23:38+00:00,Trump,597108,28830,13958,Alabama,86.9,13.1,50.1,1100.3,10298.0,
12,2020-11-04 17:16:47+00:00,Trump,586810,43805,48782,Alabama,17.7,82.3,82.3,729.3,31534.0,


In [64]:
filtered_df.dtypes

Timestamp                  datetime64[ns, UTC]
In The Lead                             object
Vote Margin                              int64
Votes Remaining (est.)                  object
Change                                  object
State                                   object
Leader Batch Breakdown                 float64
Trailer Batch Breakdown                float64
Trailer Batch Trend                     object
Trailer Hurdle                          object
Margin_change_estimate                 float64
Type                                    object
dtype: object

In [65]:
result_df = filtered_df.copy()

In [66]:
result_df[['Change','Trailer Batch Trend', 'Trailer Hurdle']] = result_df[['Change','Trailer Batch Trend', 'Trailer Hurdle']].apply(pd.to_numeric)

In [67]:
result_df.dtypes

Timestamp                  datetime64[ns, UTC]
In The Lead                             object
Vote Margin                              int64
Votes Remaining (est.)                  object
Change                                 float64
State                                   object
Leader Batch Breakdown                 float64
Trailer Batch Breakdown                float64
Trailer Batch Trend                    float64
Trailer Hurdle                         float64
Margin_change_estimate                 float64
Type                                    object
dtype: object

In [68]:
result_df.sort_values(by=['State', 'Timestamp'], ascending=[True, False], inplace=True)

In [69]:
# needed for groupby setup so we have access to index number within the df
result_df.reset_index(inplace=True, drop=True)
result_df.reset_index(inplace=True)

In [70]:
result_df.head(10)

,index,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type
0,0,2020-12-04 21:12:00+00:00,Trump,591520,Unknown,22.0,Alabama,9.1,109.1,58.5,NaN,26.0,
1,1,2020-11-23 20:33:25+00:00,Trump,591546,4,2476.0,Alabama,44.2,55.8,58.5,7499241.2,286.0,
2,2,2020-11-23 20:11:23+00:00,Trump,591832,2521,5894.0,Alabama,50.6,49.4,55.0,11954.5,72.0,
3,3,2020-11-23 19:48:16+00:00,Trump,591760,8534,4792.0,Alabama,61.2,38.8,47.8,3566.2,1074.0,
4,4,2020-11-06 22:26:37+00:00,Trump,590686,13386,12548.0,Alabama,28.6,71.4,44.9,2287.6,5360.0,
5,5,2020-11-04 21:12:19+00:00,Trump,595572,25662,10.0,Alabama,30.0,70.0,49.4,1226.9,4.0,
6,6,2020-11-04 21:00:38+00:00,Trump,595576,25815,2938.0,Alabama,59.2,40.8,49.5,1219.9,542.0,
7,7,2020-11-04 20:28:27+00:00,Trump,595034,28763,54.0,Alabama,1870.4,1970.4,53.5,1099.0,2074.0,
8,8,2020-11-04 20:23:38+00:00,Trump,597108,28830,13958.0,Alabama,86.9,13.1,50.1,1100.3,10298.0,
9,9,2020-11-04 17:16:47+00:00,Trump,586810,43805,48782.0,Alabama,17.7,82.3,82.3,729.3,31534.0,


In [71]:
len(result_df)

3060

In [72]:
last_rows = list(result_df.groupby('State').last()['index'])
print(last_rows)

[10, 26, 112, 194, 285, 353, 461, 467, 477, 494, 711, 715, 722, 785, 833, 903, 963, 1014, 1029, 1121, 1219, 1345, 1405, 1582, 1640, 1650, 1670, 1681, 1727, 1789, 1934, 1955, 2027, 2086, 2120, 2160, 2165, 2198, 2522, 2534, 2592, 2633, 2640, 2688, 2742, 2772, 2861, 3011, 3028, 3054, 3059]


In [73]:
first_rows = list(result_df.groupby('State').first()['index'])
print(first_rows)


[0, 11, 27, 113, 195, 286, 354, 462, 468, 478, 495, 712, 716, 723, 786, 834, 904, 964, 1015, 1030, 1122, 1220, 1346, 1406, 1583, 1641, 1651, 1671, 1682, 1728, 1790, 1935, 1956, 2028, 2087, 2121, 2161, 2166, 2199, 2523, 2535, 2593, 2634, 2641, 2689, 2743, 2773, 2862, 3012, 3029, 3055]


In [74]:
result_df[['Batch Votes Winner', 'Batch Votes Loser', 'Accum. Votes Winner', 'Accum. Votes Loser']] = ""
result_df.head(3)

,index,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type,Batch Votes Winner,Batch Votes Loser,Accum. Votes Winner,Accum. Votes Loser
0,0,2020-12-04 21:12:00+00:00,Trump,591520,Unknown,22.0,Alabama,9.1,109.1,58.5,NaN,26.0,,,,,
1,1,2020-11-23 20:33:25+00:00,Trump,591546,4,2476.0,Alabama,44.2,55.8,58.5,7499241.2,286.0,,,,,
2,2,2020-11-23 20:11:23+00:00,Trump,591832,2521,5894.0,Alabama,50.6,49.4,55.0,11954.5,72.0,,,,,


In [75]:
result_df[['Batch Votes Winner', 'Batch Votes Loser', 'Accum. Votes Winner', 'Accum. Votes Loser']] = result_df[['Batch Votes Winner', 'Batch Votes Loser', 'Accum. Votes Winner', 'Accum. Votes Loser']].apply(pd.to_numeric)

result_df.head(3)

,index,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type,Batch Votes Winner,Batch Votes Loser,Accum. Votes Winner,Accum. Votes Loser
0,0,2020-12-04 21:12:00+00:00,Trump,591520,Unknown,22.0,Alabama,9.1,109.1,58.5,NaN,26.0,,NaN,NaN,NaN,NaN
1,1,2020-11-23 20:33:25+00:00,Trump,591546,4,2476.0,Alabama,44.2,55.8,58.5,7499241.2,286.0,,NaN,NaN,NaN,NaN
2,2,2020-11-23 20:11:23+00:00,Trump,591832,2521,5894.0,Alabama,50.6,49.4,55.0,11954.5,72.0,,NaN,NaN,NaN,NaN


In [76]:
## result_df.drop(result_df.columns[[11]], axis='columns', inplace=True)

In [77]:
result_df.dtypes

index                                    int64
Timestamp                  datetime64[ns, UTC]
In The Lead                             object
Vote Margin                              int64
Votes Remaining (est.)                  object
Change                                 float64
State                                   object
Leader Batch Breakdown                 float64
Trailer Batch Breakdown                float64
Trailer Batch Trend                    float64
Trailer Hurdle                         float64
Margin_change_estimate                 float64
Type                                    object
Batch Votes Winner                     float64
Batch Votes Loser                      float64
Accum. Votes Winner                    float64
Accum. Votes Loser                     float64
dtype: object

In [78]:
# Need to remove the rows of change data that is smaller than the vote margin change - that data doesn't make sense
bad_data_rows = []
for i in range(len(result_df)-1):
    if i in last_rows:
        ## Skip = Not valuable data
        continue
    else:
        change = result_df.iloc[i,5]
        vote_margin_now = int(result_df.iloc[i,3])
        vote_margin_prev = int(result_df.iloc[i+1,3])
        
        if change > abs(vote_margin_now - vote_margin_prev):
            ## Skip = Good Data
            continue
        else: 
            bad_data_rows.append(i)

In [79]:
len(bad_data_rows)

384

In [80]:
bad_data_rows[0:10]

[0, 7, 12, 13, 27, 28, 32, 36, 41, 45]

In [81]:
result_df.drop(bad_data_rows, inplace=True, )

In [82]:
result_df.head(30)

,index,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type,Batch Votes Winner,Batch Votes Loser,Accum. Votes Winner,Accum. Votes Loser
1,1,2020-11-23 20:33:25+00:00,Trump,591546,4,2476.0,Alabama,44.2,55.8,58.5,7499241.2,286.0,,NaN,NaN,NaN,NaN
2,2,2020-11-23 20:11:23+00:00,Trump,591832,2521,5894.0,Alabama,50.6,49.4,55.0,11954.5,72.0,,NaN,NaN,NaN,NaN
3,3,2020-11-23 19:48:16+00:00,Trump,591760,8534,4792.0,Alabama,61.2,38.8,47.8,3566.2,1074.0,,NaN,NaN,NaN,NaN
4,4,2020-11-06 22:26:37+00:00,Trump,590686,13386,12548.0,Alabama,28.6,71.4,44.9,2287.6,5360.0,,NaN,NaN,NaN,NaN
5,5,2020-11-04 21:12:19+00:00,Trump,595572,25662,10.0,Alabama,30.0,70.0,49.4,1226.9,4.0,,NaN,NaN,NaN,NaN
6,6,2020-11-04 21:00:38+00:00,Trump,595576,25815,2938.0,Alabama,59.2,40.8,49.5,1219.9,542.0,,NaN,NaN,NaN,NaN
8,8,2020-11-04 20:23:38+00:00,Trump,597108,28830,13958.0,Alabama,86.9,13.1,50.1,1100.3,10298.0,,NaN,NaN,NaN,NaN
9,9,2020-11-04 17:16:47+00:00,Trump,586810,43805,48782.0,Alabama,17.7,82.3,82.3,729.3,31534.0,,NaN,NaN,NaN,NaN
10,10,2020-11-04 13:28:22+00:00,Trump,618344,93031,0.0,Alabama,NaN,NaN,NaN,387.0,NaN,,NaN,NaN,NaN,NaN
11,11,2020-12-02 15:22:19+00:00,Trump,36173,31816,587.0,Alaska,22.7,77.3,49.4,109.5,321.0,,NaN,NaN,NaN,NaN


In [83]:
# find if any percentages are greater than 100% and remove
df33 = result_df[ (result_df['Trailer Batch Breakdown'] > 100) | (result_df['Trailer Batch Breakdown'] < 0)]
df33.head()

,index,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type,Batch Votes Winner,Batch Votes Loser,Accum. Votes Winner,Accum. Votes Loser


In [84]:
if len(df33)>0:
    result_df.drop(list(df33.index), inplace=True)
else:
    print("No Values greater than 100")

No Values greater than 100


In [85]:
temp_store = result_df

In [86]:
result_df = temp_store

In [87]:
# Reset index so index goes from 1,2,...,N
result_df.drop('index', axis='columns', inplace=True)

In [88]:
result_df.reset_index(drop=True, inplace=True)

In [89]:
result_df.reset_index(inplace=True)
result_df.head(2)

,index,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type,Batch Votes Winner,Batch Votes Loser,Accum. Votes Winner,Accum. Votes Loser
0,0,2020-11-23 20:33:25+00:00,Trump,591546,4,2476.0,Alabama,44.2,55.8,58.5,7499241.2,286.0,,NaN,NaN,NaN,NaN
1,1,2020-11-23 20:11:23+00:00,Trump,591832,2521,5894.0,Alabama,50.6,49.4,55.0,11954.5,72.0,,NaN,NaN,NaN,NaN


In [90]:
#result_df.head(10)

In [91]:
result_df.dtypes

index                                    int64
Timestamp                  datetime64[ns, UTC]
In The Lead                             object
Vote Margin                              int64
Votes Remaining (est.)                  object
Change                                 float64
State                                   object
Leader Batch Breakdown                 float64
Trailer Batch Breakdown                float64
Trailer Batch Trend                    float64
Trailer Hurdle                         float64
Margin_change_estimate                 float64
Type                                    object
Batch Votes Winner                     float64
Batch Votes Loser                      float64
Accum. Votes Winner                    float64
Accum. Votes Loser                     float64
dtype: object

In [92]:
first_rows = list(result_df.groupby('State').first()['index'])
print(first_rows)

[0, 9, 23, 98, 171, 262, 326, 417, 421, 431, 445, 636, 640, 644, 701, 743, 803, 860, 906, 910, 1001, 1058, 1173, 1226, 1368, 1424, 1431, 1449, 1458, 1492, 1549, 1685, 1705, 1776, 1834, 1861, 1900, 1902, 1934, 2242, 2250, 2304, 2321, 2327, 2370, 2401, 2410, 2490, 2635, 2652, 2673]


In [93]:
last_rows = list(result_df.groupby('State').last()['index'])
print(last_rows)

[8, 22, 97, 170, 261, 325, 416, 420, 430, 444, 635, 639, 643, 700, 742, 802, 859, 905, 909, 1000, 1057, 1172, 1225, 1367, 1423, 1430, 1448, 1457, 1491, 1548, 1684, 1704, 1775, 1833, 1860, 1899, 1901, 1933, 2241, 2249, 2303, 2320, 2326, 2369, 2400, 2409, 2489, 2634, 2651, 2672, 2675]


In [94]:
state_index = 0
for i in range(len(result_df)-1):
    if i in first_rows:
        winner_final_votes = summary_df.iloc[state_index,3].astype(int)
        loser_final_votes = summary_df.iloc[state_index, 6].astype(int)
        
        #Accum. Votes Winner
        result_df.iloc[i,15] = winner_final_votes.astype(int)
                
        #Accum. Votes Loser
        result_df.iloc[i,16] = loser_final_votes.astype(int)
        
        # Next state
        state_index += 1
        
        # Calculate Batch Votes
        change = int(result_df.iloc[i,5])
        vote_margin_now = int(result_df.iloc[i,3])
        vote_margin_prev = int(result_df.iloc[i+1,3])
        
        # Batch Votes Loser
        bvl = ( change - vote_margin_now + vote_margin_prev )/2
        result_df.iloc[i,14] = bvl
        
        # Batch Votes Winner
        bvw = bvl + vote_margin_now - vote_margin_prev
        result_df.iloc[i,13] = bvw
        
    elif i in last_rows:
        change = int(result_df.iloc[i,5])
        loser_batch_percent = result_df.iloc[i,8]/100
        winner_batch_percent = result_df.iloc[i,7]/100
        
        # Batch Votes Loser
        result_df.iloc[i,14] = round(result_df.iloc[i-1,16] - result_df.iloc[i-1,14],0)
        
        # Batch Votes Winner
        result_df.iloc[i,13] = round(result_df.iloc[i-1,15] - result_df.iloc[i-1,13],0)

        # Accum Winner
        result_df.iloc[i,15] = round(result_df.iloc[i-1,15] - result_df.iloc[i-1,13],0)
        
        # Accum Loser
        result_df.iloc[i,16] = round(result_df.iloc[i-1,16] - result_df.iloc[i-1,14],0)
        
    else:
        change = int(result_df.iloc[i,5])
        vote_margin_now = int(result_df.iloc[i,3])
        vote_margin_prev = int(result_df.iloc[i+1,3])
        
        # Batch Votes Loser
        bvl = ( change - vote_margin_now + vote_margin_prev )/2
        result_df.iloc[i,14] = bvl
        
        # Batch Votes Winner
        bvw = bvl + vote_margin_now - vote_margin_prev
        result_df.iloc[i,13] = bvw

        # Accum Winner
        result_df.iloc[i,15] = result_df.iloc[i-1,15] - bvw
        
        # Accum Loser
        result_df.iloc[i,16] = result_df.iloc[i-1,16] - bvl

In [95]:
result_df.head(30)

,index,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type,Batch Votes Winner,Batch Votes Loser,Accum. Votes Winner,Accum. Votes Loser
0,0,2020-11-23 20:33:25+00:00,Trump,591546,4,2476.0,Alabama,44.2,55.8,58.5,7499241.2,286.0,,1095.0,1381.0,1441170.0,849624.0
1,1,2020-11-23 20:11:23+00:00,Trump,591832,2521,5894.0,Alabama,50.6,49.4,55.0,11954.5,72.0,,2983.0,2911.0,1438187.0,846713.0
2,2,2020-11-23 19:48:16+00:00,Trump,591760,8534,4792.0,Alabama,61.2,38.8,47.8,3566.2,1074.0,,2933.0,1859.0,1435254.0,844854.0
3,3,2020-11-06 22:26:37+00:00,Trump,590686,13386,12548.0,Alabama,28.6,71.4,44.9,2287.6,5360.0,,3831.0,8717.0,1431423.0,836137.0
4,4,2020-11-04 21:12:19+00:00,Trump,595572,25662,10.0,Alabama,30.0,70.0,49.4,1226.9,4.0,,3.0,7.0,1431420.0,836130.0
5,5,2020-11-04 21:00:38+00:00,Trump,595576,25815,2938.0,Alabama,59.2,40.8,49.5,1219.9,542.0,,703.0,2235.0,1430717.0,833895.0
6,6,2020-11-04 20:23:38+00:00,Trump,597108,28830,13958.0,Alabama,86.9,13.1,50.1,1100.3,10298.0,,12128.0,1830.0,1418589.0,832065.0
7,7,2020-11-04 17:16:47+00:00,Trump,586810,43805,48782.0,Alabama,17.7,82.3,82.3,729.3,31534.0,,8624.0,40158.0,1409965.0,791907.0
8,8,2020-11-04 13:28:22+00:00,Trump,618344,93031,0.0,Alabama,NaN,NaN,NaN,387.0,NaN,,1401341.0,751749.0,1401341.0,751749.0
9,9,2020-12-02 15:22:19+00:00,Trump,36173,31816,587.0,Alaska,22.7,77.3,49.4,109.5,321.0,,359.5,227.5,189951.0,153778.0


In [96]:
result_df[result_df['Batch Votes Winner'].isna()]

,index,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type,Batch Votes Winner,Batch Votes Loser,Accum. Votes Winner,Accum. Votes Loser
2675,2675,2020-11-04 13:28:22+00:00,Trump,119839,422,0.0,Wyoming,NaN,NaN,NaN,14765.5,NaN,,NaN,NaN,NaN,NaN


In [97]:
result_df[2670: 2676]

,index,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type,Batch Votes Winner,Batch Votes Loser,Accum. Votes Winner,Accum. Votes Loser
2670,2670,2020-11-04 18:30:33+00:00,Biden,20509,294,35.0,Wisconsin,68.8,31.2,82.6,3598.9,24.0,updated,5.5,29.5,1610328.5,1589869.5
2671,2671,2020-11-04 17:57:21+00:00,Biden,20533,229,352.0,Wisconsin,26.7,73.3,73.3,4611.5,164.0,,94.0,258.0,1610234.5,1589611.5
2672,2672,2020-11-04 13:28:22+00:00,Biden,20697,592,0.0,Wisconsin,NaN,NaN,NaN,1828.6,NaN,,1610140.0,1589354.0,1610140.0,1589354.0
2673,2673,2020-11-11 20:39:54+00:00,Trump,120068,84,151.0,Wyoming,69.5,30.5,15.4,74119.0,59.0,,105.0,46.0,193559.0,73491.0
2674,2674,2020-11-06 02:50:08+00:00,Trump,120009,237,180.0,Wyoming,97.2,2.8,2.8,26289.4,170.0,,175.0,5.0,193384.0,73486.0
2675,2675,2020-11-04 13:28:22+00:00,Trump,119839,422,0.0,Wyoming,NaN,NaN,NaN,14765.5,NaN,,NaN,NaN,NaN,NaN


In [98]:
result_df.loc[2675, 'Accum. Votes Winner'] = (193384 - 175)
result_df.loc[2675, 'Accum. Votes Loser'] = (73486 - 5)
result_df.loc[2675, 'Batch Votes Winner'] = (193384 - 175)
result_df.loc[2675, 'Batch Votes Loser'] = (73486 - 5)

In [99]:
result_df[2670: 2676]

,index,Timestamp,In The Lead,Vote Margin,Votes Remaining (est.),Change,State,Leader Batch Breakdown,Trailer Batch Breakdown,Trailer Batch Trend,Trailer Hurdle,Margin_change_estimate,Type,Batch Votes Winner,Batch Votes Loser,Accum. Votes Winner,Accum. Votes Loser
2670,2670,2020-11-04 18:30:33+00:00,Biden,20509,294,35.0,Wisconsin,68.8,31.2,82.6,3598.9,24.0,updated,5.5,29.5,1610328.5,1589869.5
2671,2671,2020-11-04 17:57:21+00:00,Biden,20533,229,352.0,Wisconsin,26.7,73.3,73.3,4611.5,164.0,,94.0,258.0,1610234.5,1589611.5
2672,2672,2020-11-04 13:28:22+00:00,Biden,20697,592,0.0,Wisconsin,NaN,NaN,NaN,1828.6,NaN,,1610140.0,1589354.0,1610140.0,1589354.0
2673,2673,2020-11-11 20:39:54+00:00,Trump,120068,84,151.0,Wyoming,69.5,30.5,15.4,74119.0,59.0,,105.0,46.0,193559.0,73491.0
2674,2674,2020-11-06 02:50:08+00:00,Trump,120009,237,180.0,Wyoming,97.2,2.8,2.8,26289.4,170.0,,175.0,5.0,193384.0,73486.0
2675,2675,2020-11-04 13:28:22+00:00,Trump,119839,422,0.0,Wyoming,NaN,NaN,NaN,14765.5,NaN,,193209.0,73481.0,193209.0,73481.0


In [100]:
result_df.to_csv('./data/cleaned_records.csv', index=False)

In [101]:
time_end = pd.Timestamp.now()
print(time_end)

2022-01-19 04:04:30.174822


In [102]:
print(f'Elapsed time: {time_end - time_start}')

Elapsed time: 0 days 00:00:23.489245


## Analyze Trends

### Metrics
* Need table of each column and how many valid rows, invalid rows, and calculated rows
* With data analysis -what is the difference between dropping all 'Unknown' and '0' values for the "Change" column and removal of all percentages outside 0 to 100.
* Need to show the effect of calculating the Unknown columns versus what the effect would be by dropping it.
* Need to show the residual of vote percents (vote_pred - vote_recorded) and vote margins.
* Need to show outliers by plotting the change in margin versus the change in candidate votes - are there outliers - what did this look like before cleaning

In [103]:
# df = result_df

In [104]:
# df.columns

In [105]:
# max = df['Timestamp'].max()

In [106]:
# min = df['Timestamp'].min()
# min

In [107]:
# ax = df.groupby('State').plot(y='Accum. Votes Winner', x='Timestamp', figsize=(80, 10));
# ax.set_xlim(pd.Timestamp(min), pd.Timestamp(max))

In [108]:
# df.plot(y='Accum. Votes Winner', x='Timestamp')

## Format Data for Database

Format of database  
  * batch # (use index - continuous across all states)
  * datetime
  * candidate
  * votes for candidate
  * state
  * district (empty for now)
  * type ( :: accepted, rejected, calculated)

In [109]:
batch_records = []
dict = {}
for i in range(len(result_df)):
    datetime = result_df.iloc[i,1]
    leader = result_df.iloc[i,2]
    leader_votes = result_df.iloc[i,13]
    state = result_df.iloc[i,6]
    district = ""
    type_desc = result_df.iloc[i,12]
    
    if (leader == 'Trump'):
        runner_up = 'Biden'
    else:
        runner_up = 'Trump'
    
    runner_up_votes = result_df.iloc[i,14]
    
    dict = {
        "datetime": datetime,
        "candidate": leader,
        "votes": leader_votes,
        "state": state,
        "dictrict": district,
        "type": type_desc
    }
    
    batch_records.append(dict)
    
    dict = {
        "datetime": datetime,
        "candidate": runner_up,
        "votes": runner_up_votes,
        "state": state,
        "dictrict": district,
        "type": type_desc
    }
    
    batch_records.append(dict)
    
    

In [110]:
df = pd.DataFrame(batch_records)
df

,datetime,candidate,votes,state,dictrict,type
0,2020-11-23 20:33:25+00:00,Trump,1095.0,Alabama,,
1,2020-11-23 20:33:25+00:00,Biden,1381.0,Alabama,,
2,2020-11-23 20:11:23+00:00,Trump,2983.0,Alabama,,
3,2020-11-23 20:11:23+00:00,Biden,2911.0,Alabama,,
4,2020-11-23 19:48:16+00:00,Trump,2933.0,Alabama,,
...,...,...,...,...,...,...
5347,2020-11-11 20:39:54+00:00,Biden,46.0,Wyoming,,
5348,2020-11-06 02:50:08+00:00,Trump,175.0,Wyoming,,
5349,2020-11-06 02:50:08+00:00,Biden,5.0,Wyoming,,
5350,2020-11-04 13:28:22+00:00,Trump,193209.0,Wyoming,,


In [111]:
df.dtypes

datetime     datetime64[ns, UTC]
candidate                 object
votes                    float64
state                     object
dictrict                  object
type                      object
dtype: object

In [112]:
df.describe()

,votes
count,5.352000e+03
mean,2.892839e+04
std,2.599932e+05
min,-1.018950e+04
25%,5.600000e+01
50%,4.750000e+02
75%,2.876750e+03
max,7.721336e+06


In [113]:
df['votes'].value_counts()

1.0        103
2.0         85
4.0         65
5.0         65
3.0         63
          ... 
298.0        1
75598.0      1
435.0        1
7497.0       1
1570.0       1
Name: votes, Length: 2803, dtype: int64

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5352 entries, 0 to 5351
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   datetime   5352 non-null   datetime64[ns, UTC]
 1   candidate  5352 non-null   object             
 2   votes      5352 non-null   float64            
 3   state      5352 non-null   object             
 4   dictrict   5352 non-null   object             
 5   type       5352 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(1), object(4)
memory usage: 251.0+ KB


In [115]:
df[df['votes'].isna()]
# looks like 2 nan values out of 5352 records
# these are the first record for each state and the district of columbia that don't have a calculated candidate votes

,datetime,candidate,votes,state,dictrict,type
